In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [ ]:
def extract_time(row):
    # Convert to pacific time
    date = datetime.strptime(row.date, '%Y-%m-%dT%H:%M:%S.%fZ') + timedelta(hours=-7)
    return date.time().replace(second=0, microsecond=0)

def extract_seconds(row):
    time = row.time
    return (time.hour * 60 + time.minute)

votes = pd.read_csv("all_extrap.csv")
votes = votes.set_index('post_id')
# votes['time'] = votes.apply(lambda row: extract_time(row), axis=1)
# votes['second'] = votes.apply(lambda row: extract_seconds(row), axis=1)
votes['score_2'] = votes.apply(lambda row: row.score * row.score, axis=1)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [ ]:
def train_model(votes, from_band, to_band):
    data = votes[(votes.band == from_band)].join(votes[(votes.band == to_band)].score, how='inner', rsuffix='_final')
    train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)
#     lin_reg = MLPRegressor(hidden_layer_sizes=(10,10))
    lin_reg = LinearRegression()
    learn_from = ['score']
    lin_reg.fit(train_set[learn_from], train_set.score_final.values.reshape(-1, 1))
    predictions = lin_reg.predict(test_set[learn_from])
    return {
        'coeff': [lin_reg.intercept_[0], lin_reg.coef_[0][0]],
        'rms': np.sqrt(mean_squared_error(test_set.score_final.values.reshape(-1, 1), predictions))
    } 

In [ ]:
all_predictions = []
# 444697.48566902045
train_from = votes #[votes.scoreAt24h < 500]
error = 0

for x in range(288):
    predictions = []
#     print("Processing " + str(x))
    for y in range(x + 1, 288):
        res = train_model(train_from, x, y)
        predictions.append(res['coeff']) 
        error += res['rms']

    all_predictions.append(predictions)
    
print("Error is ", error)

In [ ]:
import json
with open('coeff_interp1.json', 'w') as outfile:
    json.dump(all_predictions, outfile)

In [ ]:
from sklearn.model_selection import cross_val_score
lin_reg = LinearRegression()
learn_from = ['score']
scores = cross_val_score(lin_reg, data[learn_from], data.score_final.values.reshape(-1, 1), scoring="neg_mean_squared_error", cv=10)
scores = np.sqrt(-scores)
print ("Scores: ", scores)
print ("Mean: ", scores.mean())
print ("std dev: ", scores.std())

In [ ]:
votes[(votes.band == 1) & (votes.scoreAt24h > 5000)].score.describe()

In [ ]:
votes[(votes.scoreAt24h > 5000) & (votes.band == 287)]

In [ ]:
votes[votes.index == '/r/dataisbeautiful/comments/csl706/i_recorded_my_travels_as_a_professional_truck/'][['band', 'score']].to_json('values.json', orient='values')

In [ ]:
votes.loc[votes[(votes.band == 35) & (votes.score < 50) & (votes.scoreAt24h > 8000)].index].plot(x='band', y='score', figsize=(15,5), style='.')

In [ ]:
votes.loc[votes[(votes.band == 35) & (votes.score < 50) & (votes.scoreAt24h > 8000)].index]

In [ ]:
pd.read_csv("all_perfect.csv").plot(x='band', y='score', style='.', figsize=(15,10))

In [ ]:
pd.read_csv("all.csv").plot(x='band', y='score', style='.', figsize=(15,10))

In [ ]:
pd.read_csv("all_extrap.csv").plot(x='band', y='score', style='.', figsize=(15,10))

In [ ]:
votes[votes.band==1].hist(bins=12, figsize=(15,10))

In [ ]:
votes[(votes.band==1) &(votes.score>20)]